In [ ]:
import tensorflow as tf

train_dir = "./data/train"
valid_dir = "./data/valid"
test_dir  = "./data/test"

img_size = (224, 224)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir, 
    image_size=img_size, 
    batch_size=batch_size
)
valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
    valid_dir, 
    image_size=img_size, 
    batch_size=batch_size
)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir, 
    image_size=img_size, 
    batch_size=batch_size
)


Found 70295 files belonging to 38 classes.
Found 17572 files belonging to 38 classes.
Found 33 files belonging to 1 classes.


In [8]:
from tensorflow.keras import layers
# Chuẩn hóa và augmentation
data_augmentation = tf.keras.Sequential([
    layers.Rescaling(1./255),
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

In [9]:
from tensorflow.keras import applications
base_model = applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

model = tf.keras.Sequential([
    data_augmentation,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(38, activation="softmax")  # 38 lớp bệnh
])

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [10]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_4 (Sequential)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ ?                      │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,257,984 (8.61 MB)

In [11]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

checkpoint = ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_accuracy", mode="max")
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

history = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=20,
    callbacks=[checkpoint, early_stop]
)


Epoch 1/20
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 1404s 638ms/step - accuracy: 0.8597 - loss: 0.4881 - val_accuracy: 0.9146 - val_loss: 0.2679
Epoch 2/20
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 1363s 620ms/step - accuracy: 0.9194 - loss: 0.2478 - val_accuracy: 0.9237 - val_loss: 0.2311
Epoch 3/20
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 1357s 618ms/step - accuracy: 0.9249 - loss: 0.2231 - val_accuracy: 0.9334 - val_loss: 0.2029
Epoch 4/20
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 1326s 604ms/step - accuracy: 0.9308 - loss: 0.2058 - val_accuracy: 0.9346 - val_loss: 0.1934
Epoch 5/20
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 1316s 599ms/step - accuracy: 0.9338 - loss: 0.1994 - val_accuracy: 0.9346 - val_loss: 0.1906
Epoch 6/20
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 1310s 596ms/step - accuracy: 0.9360 - loss: 0.1908 - val_accuracy: 0.9381 - val_loss: 0.1944
Epoch 7/20
1915/2197 ━━━━━━━━━━━━━━━━━━━━ 2:15 482ms/step - accuracy: 0.9350 - loss: 0.1879

KeyboardInterrupt: 